In [2]:
import pagerank
from TextRank import textrank

In [3]:
text = "This module implements TextRank, an unsupervised keyword significance scoring algorithm."
relevant_pos_tag = ["NN", "ADJ"]

In [4]:
words = textrank.__preprocess_document(text, relevant_pos_tag)
print(words)

['module', 'keyword', 'significance', 'algorithm']


In [5]:
import collections
window_size = 2

In [6]:
edge_weights = collections.defaultdict(lambda: collections.Counter())
for index, word in enumerate(words):
    for other_index in range(index - window_size, index + window_size + 1):
        if other_index >= 0 and other_index < len(words) and other_index != index:
            other_word = words[other_index]
            edge_weights[word][other_word] += 1.0
print(edge_weights)

defaultdict(<function <lambda> at 0x00000130D35EF920>, {'module': Counter({'keyword': 1.0, 'significance': 1.0}), 'keyword': Counter({'module': 1.0, 'significance': 1.0, 'algorithm': 1.0}), 'significance': Counter({'module': 1.0, 'keyword': 1.0, 'algorithm': 1.0}), 'algorithm': Counter({'keyword': 1.0, 'significance': 1.0})})


In [7]:
import pandas

In [8]:
transition_weights = pandas.DataFrame(edge_weights)
print(transition_weights)

              module  keyword  significance  algorithm
keyword          1.0      NaN           1.0        1.0
significance     1.0      1.0           NaN        1.0
module           NaN      1.0           1.0        NaN
algorithm        NaN      1.0           1.0        NaN


In [9]:
nodes = pagerank.__extract_nodes(transition_weights)
print(nodes)

{'significance', 'keyword', 'algorithm', 'module'}


In [10]:
transition_weights = pagerank.__make_square(transition_weights, nodes, default=0.0)
transition_weights = pagerank.__ensure_rows_positive(transition_weights)
print(transition_weights)

              module  keyword  significance  algorithm
keyword          1.0      0.0           1.0        1.0
significance     1.0      1.0           0.0        1.0
module           0.0      1.0           1.0        0.0
algorithm        0.0      1.0           1.0        0.0


In [11]:
transition_weights = pagerank.__normalize_rows(transition_weights)
print(transition_weights)

                module   keyword  significance  algorithm
keyword       0.333333  0.000000      0.333333   0.333333
significance  0.333333  0.333333      0.000000   0.333333
module        0.000000  0.500000      0.500000   0.000000
algorithm     0.000000  0.500000      0.500000   0.000000


random surfer moves some probabilities from connected nodes to other unconnected nodes

In [12]:
transition_weights = pagerank.__integrate_random_surfer(nodes, transition_weights, rsp=0.15)
print(transition_weights)

                module   keyword  significance  algorithm
keyword       0.320833  0.037500      0.320833   0.320833
significance  0.320833  0.320833      0.037500   0.320833
module        0.037500  0.462500      0.462500   0.037500
algorithm     0.037500  0.462500      0.462500   0.037500


In [13]:
state = pagerank.__start_state(nodes)
print(state)

significance    0.25
keyword         0.25
algorithm       0.25
module          0.25
dtype: float64


In [14]:
norm = pagerank.__euclidean_norm(state)
print(norm)

0.5


Formula of textRank from [paper](https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf):

$WS(V_i) = (1 - d) + d * \sum_{V_j \in \text{In}(V_i)} \frac{w_{ji}}{\sum_{V_k \in \text{Out}(V_j)}w_{jk}}WS(V_j)$

It can be seen as a markov chain, and for an aperiodic irreducible markov chain, repeating the iterative process above is guaranteed to converge to a stable point.

Some notes:
* the role of $d$ is taken care of in the *__integrate_random_surfer* function
* since we have normalized the edge weights with *__normalize_rows* function, $\sum_{V_k \in \text{Out}(V_j)} w_{jk}$ sums to $1$. 

In [15]:
max_iterations = 1000
epsilon = 0.00001

In [16]:
# Power iteration:
for iteration in range(max_iterations):
    old_state = state.copy()
    state = state.dot(transition_weights)
    delta = state - old_state
    if pagerank.__euclidean_norm(delta) < epsilon: 
        break

In [44]:
print("break at iteration: ", iteration)
print("~" * 10 + " 2nd last " + "~" * 10)
print(old_state)
print("~" * 10 + " last " + "~" * 10)
print(state)

break at iteration:  15
~~~~~~~~~~ 2nd last ~~~~~~~~~~
module          0.204785
keyword         0.295215
significance    0.295215
algorithm       0.204785
dtype: float64
~~~~~~~~~~ last ~~~~~~~~~~
module          0.204788
keyword         0.295212
significance    0.295212
algorithm       0.204788
dtype: float64


state initialization only affects convergence speed, but not convergence itself

In [38]:
import random

In [46]:
initial_state_vals = [random.random() for i in range(4)]
initial_state_vals = [val / sum(initial_state_vals) for val in initial_state_vals]
state = pandas.Series(initial_state_vals, index=state.index)
print(state)

module          0.417535
keyword         0.048822
significance    0.220407
algorithm       0.313236
dtype: float64


In [47]:
for iteration in range(max_iterations):
    old_state = state.copy()
    state = state.dot(transition_weights)
    delta = state - old_state
    if pagerank.__euclidean_norm(delta) < epsilon: 
        break

print("break at iteration: ", iteration)
print("~" * 10 + " 2nd last " + "~" * 10)
print(old_state)
print("~" * 10 + " last " + "~" * 10)
print(state)

break at iteration:  20
~~~~~~~~~~ 2nd last ~~~~~~~~~~
module          0.204789
keyword         0.295211
significance    0.295211
algorithm       0.204789
dtype: float64
~~~~~~~~~~ last ~~~~~~~~~~
module          0.204786
keyword         0.295214
significance    0.295214
algorithm       0.204786
dtype: float64
